In [1]:
#module importing and class definitions
import pandas as pd
from PIL import Image
from PIL import ImageOps
import os

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch
from torch import nn

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path)        #PIL image opener
        image = ImageOps.grayscale(image)   #convert to grayscale to follow tutorial
        #image = pil_to_tensor(image)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label



class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128*72, 128*72),
            nn.ReLU(),
            nn.Linear(128*72, 1028),
            nn.ReLU(),
            nn.Linear(1028, 1028),
            nn.ReLU(),
            nn.Linear(1028, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [2]:
#creating the dataloaders and instance of the network
img_dir = './all_frames/'
training_data = CustomImageDataset('training.csv', img_dir, transform=ToTensor())
testing_data = CustomImageDataset('testing.csv', img_dir, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)

model = NeuralNetwork()

In [ ]:
#testing
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]

In [3]:
#training and testing loops

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (torch.argmax(pred, 1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


learning_rate = 1e-3
epochs = 15

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(testing_dataloader, model, loss_fn)

print('Done!')


Epoch 1
-------------------
loss: 0.698549  [   64/ 1374]
loss: 0.693754  [  704/ 1374]
loss: 0.685834  [ 1344/ 1374]
Test Error: 
 Accuracy: 54.8%, Avg loss: 0.688687 

Epoch 2
-------------------
loss: 0.687853  [   64/ 1374]
loss: 0.689237  [  704/ 1374]
loss: 0.689889  [ 1344/ 1374]
Test Error: 
 Accuracy: 54.8%, Avg loss: 0.688846 

Epoch 3
-------------------
loss: 0.680543  [   64/ 1374]
loss: 0.687862  [  704/ 1374]
loss: 0.696640  [ 1344/ 1374]
Test Error: 
 Accuracy: 54.8%, Avg loss: 0.687426 

Epoch 4
-------------------
loss: 0.690059  [   64/ 1374]
loss: 0.689064  [  704/ 1374]
loss: 0.683949  [ 1344/ 1374]
Test Error: 
 Accuracy: 54.8%, Avg loss: 0.688548 

Epoch 5
-------------------
loss: 0.689210  [   64/ 1374]
loss: 0.689697  [  704/ 1374]
loss: 0.682153  [ 1344/ 1374]
Test Error: 
 Accuracy: 54.8%, Avg loss: 0.685660 

Epoch 6
-------------------
loss: 0.685078  [   64/ 1374]
loss: 0.685436  [  704/ 1374]
loss: 0.695085  [ 1344/ 1374]
Test Error: 
 Accuracy: 54.8%, A